In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, window, lit, split

spark = SparkSession.builder.appName("RealTimeWordCount").getOrCreate()

# Create a DataFrame representing the streaming data
lines_df = (
    spark.readStream.format("socket")
    .option("host", "localhost")
    .option("port", 3000)
    .load()
)

print(lines_df)

# Split lines into words and explode into separate rows
# words_df = lines_df.select(explode(split(lines_df, " ")).alias("word"))
# words_df = lines_df.select(explode(lines_df.value.split(" ")).alias("word"))
words_df = lines_df.withColumn("words", explode(split(lines_df.value, " "))) \
               .select("words")
# Count word occurrences in tumbling windows of 1 second
# word_counts_df = words_df \
#     .withColumn("count", lit(1)) \
#     .groupBy(window("word", "1 second"), "word") \
#     .agg(sum("count").alias("count"))

# Display the word counts (replace with your desired output)
words_df.writeStream.outputMode("append").format("console").start().awaitTermination()


DataFrame[value: string]
